In [10]:
import pandas as pd
import numpy as np

data = pd.read_excel("test.xlsx")
data = data.drop(
    columns=[
        "dtime",
        "packet_id",
        "packet_size",
        "cum_size",
        "delta_duration",
        "location_id",
    ]
)

In [12]:
data.head()

,unit_id,cum_packets_recv,cum_duration,batch
0,665940,1,29,1
1,665940,2,35,1
2,665940,3,40,1
3,665940,4,45,1
4,665940,5,985,1


In [13]:
only10folddata = data[data["cum_packets_recv"] % 10 == 0]

In [24]:
only10folddata.tail()

,unit_id,cum_packets_recv,cum_duration,batch
9418,665940,850,88333,10
9428,665940,860,89024,10
9438,665940,870,89146,10
9448,665940,880,91008,10
9458,665940,890,91991,10


In [15]:
datagroup = only10folddata.groupby("batch")

In [25]:
type(datagroup)

pandas.core.groupby.generic.DataFrameGroupBy

In [26]:
df = datagroup.get_group(1)
print(datagroup.get_group(1))
top_row = pd.DataFrame({"cum_packets_recv": [0], "cum_duration": [0]})
# Concat with old DataFrame and reset the Index.
df = pd.concat([top_row, df]).reset_index(drop=True)
df["cum_duration"] = pd.to_numeric(df["cum_duration"])

     unit_id  cum_packets_recv  cum_duration  batch
9     665940                10          6083      1
19    665940                20         10979      1
29    665940                30         12121      1
39    665940                40         12243      1
49    665940                50         13998      1
..       ...               ...           ...    ...
959   665940               960         92011      1
969   665940               970         92131      1
979   665940               980         92254      1
989   665940               990         93101      1
999   665940              1000         93223      1

[100 rows x 4 columns]


In [27]:
df.head()

,cum_packets_recv,cum_duration,unit_id,batch
0,0,0,NaN,NaN
1,10,6083,665940.0,1.0
2,20,10979,665940.0,1.0
3,30,12121,665940.0,1.0
4,40,12243,665940.0,1.0


In [28]:
difference = df["cum_duration"]
difference = difference.diff(periods=3)
df1 = difference.dropna()

In [30]:
df1

3      12121.0
4       6160.0
5       3019.0
6       3004.0
7       3005.0
        ...   
96      1976.0
97      1091.0
98      1091.0
99      1090.0
100     1092.0
Name: cum_duration, Length: 98, dtype: float64

In [32]:
df2 = df1.quantile(0.7)
df1 = df1.to_numpy()
mbps = 1400 * 8 * 30 / df2
df1 = np.append(df1, df2)
df1 = np.append(df1, mbps)

In [33]:
df1

array([12121.        ,  6160.        ,  3019.        ,  3004.        ,
        3005.        ,  1372.        ,  2361.        ,  4307.        ,
        5183.        ,  4195.        ,  2243.        ,  1368.        ,
         362.        ,   368.        ,   367.        ,   367.        ,
       10423.        , 10421.        , 12191.        ,  2130.        ,
        2856.        ,  1085.        ,  1864.        ,  1140.        ,
        1927.        ,  1152.        ,  1152.        ,  2039.        ,
        2035.        ,  2746.        ,  1073.        ,  4802.        ,
        4087.        ,  4088.        ,   363.        ,   370.        ,
         773.        ,  1646.        ,  1643.        ,  1962.        ,
        1089.        ,  1839.        ,  2449.        ,  3328.        ,
        2573.        ,  1240.        ,  1111.        ,  1116.        ,
        1861.        ,  1111.        ,  2759.        ,  2900.        ,
        3847.        ,  2199.        ,  1313.        ,  1024.        ,
      

In [34]:
df2

2967.9

In [7]:
df = pd.DataFrame(df1)
filepath = r"D:\my_excel_file.xlsx"
with pd.ExcelWriter(filepath, mode="a") as writer:
    df.to_excel(filepath, index=False)

ValueError: Append mode is not supported with xlsxwriter!

# Added Code 

In [37]:
# Load in data
import pandas as pd
import numpy as np

data = pd.read_excel("test.xlsx")

In [38]:
# Preprocess
data = data.drop(
    columns=[
        "dtime",
        "packet_id",
        "packet_size",
        "cum_size",
        "delta_duration",
        "location_id",
    ]
)
only10folddata = data[data["cum_packets_recv"] % 10 == 0]
datagroup = only10folddata.groupby("batch")

In [52]:
# Get groups
groups = datagroup.groups.keys()

# Create a list to hold our results:
output_list = []

# perform operations on each group:
for group in groups:
    df = datagroup.get_group(group)

    # Format the df
    top_row = pd.DataFrame({"cum_packets_recv": [0], "cum_duration": [0]})
    # Concat with old DataFrame and reset the Index.
    df = pd.concat([top_row, df]).reset_index(drop=True)
    df["cum_duration"] = pd.to_numeric(df["cum_duration"])

    # Get the
    difference = df["cum_duration"]
    difference = difference.diff(periods=3)
    df1 = difference.dropna()

    df2 = df1.quantile(0.7)
    df1 = df1.to_numpy()
    mbps = 1400 * 8 * 30 / df2

    output_list.append([group, mbps])

In [53]:
output_list

[[1, 113.211361568786],
 [2, 113.46368149123695],
 [3, 98.21689564454837],
 [4, 118.97174421074996],
 [5, 96.62947198895664],
 [6, 113.7479264700904],
 [7, 85.3398354160317],
 [8, 113.28388401888064],
 [9, 108.99182561307902],
 [10, 105.15773660490743]]

In [57]:
# If you want it in a dataframe:
output_df = pd.DataFrame(output_list, columns=["Batch", "mbps"])

In [58]:
output_df

,Batch,mbps
0,1,113.211362
1,2,113.463681
2,3,98.216896
3,4,118.971744
4,5,96.629472
5,6,113.747926
6,7,85.339835
7,8,113.283884
8,9,108.991826
9,10,105.157737
